In [1]:
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
import polars as pl
import implicit
import random
import numpy as np


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
items_meta = pl.read_parquet("items_meta.parquet")
users_meta = pl.read_parquet("users_meta.parquet")


In [3]:
test_pairs = pl.read_csv('test_pairs.csv')


test = test_pairs.with_columns(
    pl.col("user_id").cast(pl.UInt32),pl.col("item_id").cast(pl.UInt32)
)

In [4]:
it = items_meta.select("item_id","source_id","duration")

In [5]:
from sklearn.metrics import roc_auc_score
random.seed(0)
np.random.seed(0)
train = pl.scan_parquet("train_interactions.parquet")

train = train.collect()
train = train.join(users_meta,on='user_id').join(it,on = 'item_id')
train = train.with_columns(((((pl.col('share')*pl.col('like')).sum()/pl.col("share").sum()).fill_nan(0) - ((pl.col('share')*pl.col('dislike')).sum()/pl.col("share").sum()).fill_nan(0))*pl.col('share')).over(pl.col("user_id")).alias("share_user"))
train = train.with_columns(((((pl.col('share')*pl.col('like')).sum()/pl.col("share").sum()).fill_nan(0) - ((pl.col('share')*pl.col('dislike')).sum()/pl.col("share").sum()).fill_nan(0))*pl.col('share')).over(pl.col("item_id")).alias("share_item"))
train = train.with_columns(((((pl.col('bookmarks')*pl.col('like')).sum()/pl.col("bookmarks").sum()).fill_nan(0) - ((pl.col('bookmarks')*pl.col('dislike')).sum()/pl.col("bookmarks").sum()).fill_nan(0))*pl.col('bookmarks')).over(pl.col("user_id")).alias("bookmarks_user"))
train = train.with_columns(((((pl.col('bookmarks')*pl.col('like')).sum()/pl.col("bookmarks").sum()).fill_nan(0) - ((pl.col('bookmarks')*pl.col('dislike')).sum()/pl.col("bookmarks").sum()).fill_nan(0))*pl.col('bookmarks')).over(pl.col("item_id")).alias("bookmarks_item"))
train = train.with_columns((pl.corr(pl.col("like"),pl.col("timespent")/pl.col("duration")).fill_nan(0)*pl.col("timespent")/pl.col("duration")/np.abs(pl.col("timespent")/pl.col("duration")).max() - pl.corr(pl.col("dislike"),pl.col("timespent")/pl.col("duration")).fill_nan(0)*pl.col("timespent")/pl.col("duration")/np.abs(pl.col("timespent")/pl.col("duration")).max()).over(pl.col("user_id")).alias("timespent_rel_user"))
# train = train.with_columns((pl.corr(pl.col("like"),pl.col("timespent")/pl.col("duration")).fill_nan(0)*pl.col("timespent")/pl.col("duration")/np.abs(pl.col("timespent")/pl.col("duration")).max() - pl.corr(pl.col("dislike"),pl.col("timespent")/pl.col("duration")).fill_nan(0)*pl.col("timespent")/pl.col("duration")/np.abs(pl.col("timespent")/pl.col("duration")).max()).over(pl.col("item_id")).alias("timespent_rel_item"))
# train = train.with_columns((pl.corr(pl.col("like"),pl.col("timespent")).fill_nan(0)*pl.col('timespent')/np.abs(pl.col('timespent')).max() - pl.corr(pl.col("dislike"),pl.col("timespent")).fill_nan(0)*pl.col('timespent')/np.abs(pl.col('timespent')).max()).over(pl.col("user_id")).alias("timespent_abs_user"))
train = train.with_columns((pl.corr(pl.col("like"),pl.col("timespent")).fill_nan(0)*pl.col('timespent')/np.abs(pl.col('timespent')).max() - pl.corr(pl.col("dislike"),pl.col("timespent")).fill_nan(0)*pl.col('timespent')/np.abs(pl.col('timespent')).max()).over(pl.col("item_id")).alias("timespent_abs_item"))
train = train.with_columns(((pl.col("bookmarks_user")+pl.col("bookmarks_item")+pl.col("share_user")+pl.col("share_item")+pl.col("timespent_rel_user")+pl.col("timespent_abs_item"))/6).alias("targets_"))
train = train.drop(["timespent","timespent_rel_user","bookmarks_user","share_user","timespent_abs_item","bookmarks_item","share_item"])
train = train.with_columns((pl.col("targets_")+pl.col("like")*2-pl.col("dislike")*0.5).alias("targets_"))
print(roc_auc_score(train['like'],train['targets_']))
print(roc_auc_score(train['dislike'],train['targets_']))
train,valid = train_test_split(train,test_size=0.2)
import gc
gc.collect()

1.0
1.3192737178588155e-12


0

In [6]:
train = pl.concat((train,valid.filter(pl.col("user_id").is_in(valid.group_by("user_id").sum().filter((pl.col("like") ==0) | (pl.col("dislike") == 0))['user_id']))))
valid = valid.filter(pl.col("user_id").is_in(valid.group_by("user_id").sum().filter((pl.col("like") !=0) & (pl.col("dislike") != 0))['user_id']))

In [9]:
train = train.filter((pl.col("user_id").is_in(test['user_id'])) | (pl.col("item_id").is_in(test['item_id'])))
valid = valid.filter(pl.col("user_id").is_in(test['user_id']))

In [10]:
zeros = valid.filter(pl.col("user_id").is_in(train.group_by("user_id").sum().filter((pl.col("like") ==0) & (pl.col("dislike") == 0))['user_id']))
valid = valid.filter(pl.col("user_id").is_in(train.group_by("user_id").sum().filter((pl.col("like") !=0) | (pl.col("dislike") != 0))['user_id']))
train = pl.concat((train,zeros))




In [11]:
import torch
from torch import nn
from transformers import BertModel, BertConfig
from time import time

config = BertConfig(vocab_size=1,hidden_size = 32,num_attention_heads=4,num_hidden_layers=66,intermediate_size=128,max_position_embeddings=6) ##

    

class Mask(nn.Module):
    def __init__(self,p = 0.1,deviation = 0,min = 0,max = torch.inf,region = 'const'):
        super().__init__()
        self.mask_dropout = nn.Dropout(p)
        self.region = region
        assert self.region in ('const','fixed','proc')
        self.deviation = deviation
        self.min = min
        self.max = max
    def forward(self,x):
        if self.region == 'proc':
            region = torch.Tensor(x.detach().cpu()*np.random.choice(np.linspace(-self.deviation,self.deviation,100),x.shape)).to(x.device)
        elif self.region == 'const':
            region = torch.Tensor(np.random.choice(np.arange(-self.deviation,self.deviation+1),x.shape)).to(x.device)
        else:
            region = torch.Tensor(np.random.choice(np.arange(self.min,self.max+1),x.shape)).to(x.device)
            return ((self.mask_dropout(x.float()).bool().long())*x+(1-self.mask_dropout(x.float()).bool().long())*region).clamp(self.min,self.max)
        out = (x+(1-self.mask_dropout(x.float()).bool().long())*region).clamp(self.min,self.max)
        return out
        

class TaskEmbedding(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.intermidiate = nn.Linear(config.hidden_size,config.intermediate_size)
        self.act = nn.GELU()
        self.norm = nn.LayerNorm(config.intermediate_size)
        self.dropout = nn.Dropout(1)
    def forward(self,x):
        return self.norm(self.dropout(self.act(self.intermidiate(x).unsqueeze(1)))).squeeze(1)
    


class Classifier(nn.Module):
    def __init__(self,config,output_size = 1):
        super().__init__()
        self.q = nn.Linear(config.hidden_size,config.hidden_size)
        self.k = nn.Linear(config.hidden_size,config.hidden_size)
        self.v = nn.Linear(config.hidden_size,config.hidden_size)

        self.dense = nn.Linear(config.hidden_size,output_size)

        self.dropout = nn.Dropout(1)
        self.norm = nn.BatchNorm1d(config.hidden_size)
    def forward(self,x1,x2):

        x2 = x2.squeeze(2)
        x1 = x1.unsqueeze(1)
        attention = self.dropout(self.norm((torch.softmax((self.q(x1))@(self.k(x2).permute(0,2,1)),-1)@(self.v(x2))).squeeze(1)))
        return self.dense(attention)


class Recommender(nn.Module):
    def __init__(self,device = None,config = config):
        super().__init__()
        if device is not None:
            self.device = device
        else:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.config = config
        self.bert = BertModel._from_config(self.config).to(self.device)
        self.bert.embeddings.word_embeddings = nn.Dropout(0.1)
        self.users_embeddings = nn.Embedding(users_meta.shape[0],config.hidden_size).to(self.device)
        self.item_embeddings = nn.Embedding(items_meta.shape[0],config.hidden_size).to(self.device)
        self.i_feats = nn.Embedding(items_meta.shape[0],config.hidden_size).to(self.device)
        self.i_feats.requires_grad_(False)
        self.i_feats.weight.copy_(torch.Tensor(items_meta['embeddings'])).to(self.device)
        self.i_feats.requires_grad_(True)
        self.i_feats.weight.retain_grad()
        

        self.item_norm = nn.BatchNorm1d(self.config.hidden_size).to(self.device)
        self.source_norm = nn.BatchNorm1d(self.config.hidden_size).to(self.device)


        self.source_embeddings = nn.Embedding(19614,config.hidden_size-1).to(self.device)

        self.age_embeddings = nn.Embedding(120,config.hidden_size).to(self.device)
        self.gender_embeddings = nn.Embedding(3,config.hidden_size).to(self.device)

        self.age_dropout = Mask(deviation=2,p=0.2,region='const').to(self.device)
        self.gender_dropout = Mask(p = 0.01,min = 0,max = 2,region='fixed').to(self.device)
        self.source_dropout = Mask(p = 0.01,min=0,max=0,region='fixed').to(self.device)
        self.duration_dropout = Mask(p = 0.4,deviation=0.2,min=0,region='proc').to(self.device)

        self.like_head = Classifier(config).to(self.device)






    
    def forward(self,pairs,users_meta,items_meta):
        u = self.users_embeddings(torch.Tensor(pairs['user_id']).to(self.device).long())
        u_a = self.age_embeddings(self.age_dropout(torch.Tensor(users_meta["age"]).to(self.device)).long())
        u_g = self.gender_embeddings(self.gender_dropout(torch.Tensor(users_meta['gender']).to(self.device)).long())
        uu = torch.cat((u.unsqueeze(1),u_a.unsqueeze(1),u_g.unsqueeze(1)),dim=1)

        i = self.item_norm(self.item_embeddings(torch.Tensor(pairs['item_id']).to(self.device).long()))
        i_f = self.i_feats(torch.Tensor(pairs['item_id']).to(self.device).long())
        i_s = self.source_embeddings(self.source_dropout(torch.Tensor(items_meta['source_id']).to(self.device)+1).long()) ## add one to free zeroeth for unknown
        i_d = self.duration_dropout(torch.Tensor(items_meta['duration']).to(self.device)).float().unsqueeze(-1)

        i_s = self.source_norm(torch.cat((i_s,i_d),dim=-1))
        ii = torch.cat((i.unsqueeze(1),i_s.unsqueeze(1),i_f.unsqueeze(1)),dim=1)

        output = self.bert(inputs_embeds = torch.cat((uu,ii),dim=1).float(),token_type_ids = torch.cat((torch.zeros(uu.shape[0],uu.shape[1]),torch.ones(ii.shape[0],ii.shape[1])),dim=1).to(self.device).long())['last_hidden_state']
        


        l_out =  self.like_head(output[:,0,:],output)


        return None,l_out,None,None,None



        
    

In [12]:
gc.collect()
torch.cuda.empty_cache()


In [13]:
model = Recommender('cuda').to('cuda')

In [1]:
from torch.utils.data import Dataset,DataLoader
import os
from tqdm import tqdm
os.environ["POOL_SIZE"] = "24"
BATCH_SIZE = 16000
SAMPLE_SIZE = 32
class DS_Train(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data = data.partition_by("user_id", maintain_order=True)
        self.data_neg = dict()
        self.data_pos = dict()
        self.data_neutral = dict()
        
        for i in tqdm(range(len(self.data)),desc = 'Building DS'):
            udata = self.data[i]
            self.data_neg[i] = udata.filter(pl.col("like") == 0)
            self.data_pos[i] = udata.filter(pl.col("like") == 1)
            self.data_neutral[i] = udata.filter(pl.col("dislike") == 0).filter(pl.col("like") == 0)
    
    def __len__(self):
        return len(self.data)
        return 100
    
    def __getitem__(self, index):
        rand = random.randint(0,len(self.data)-1)
        data = self.data[rand]
        data = self.data[rand].sample(min(len(data),200))
        if len(self.data_pos[index] > 0 ):
            pos = self.data_pos[index].sample(min(len(self.data_pos[index]),200))
            data = pl.concat((data,pos))
        if len(self.data_neg[index] > 0 ):
            neg = self.data_neg[index].sample(min(len(self.data_neg[index]),200))
            data = pl.concat((data,neg))
        if len(self.data_neutral[index] > 0):
            neutral = self.data_neutral[index].sample(min(len(self.data_neutral[index]),200))
            data = pl.concat((data,neutral))
        return data



class DS(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data = data.partition_by("user_id", maintain_order=True)

    
    def __len__(self):
        return len(self.data)

    
    def __getitem__(self, index):
        return self.data[index]



def collate_fn(data):
    sizes = []
    for i in data:
        sizes.append(i.shape[0])
    data = pl.concat(data)
    pairs = data.select("user_id","item_id")
    users_meta = data.select("gender","age")
    items_meta = data.select("source_id",	"duration")
    targets = data.select('like','dislike','targets_')
    return pairs,users_meta,items_meta,targets,sizes





In [ ]:
train_set = DS_Train(train)
valid_set = DS(valid)

train_loader = DataLoader(train_set,batch_size=4,shuffle = True,collate_fn=collate_fn,pin_memory=True)
valid_loader = DataLoader(valid_set,batch_size = 64, shuffle = False,collate_fn = collate_fn,pin_memory=True)

In [29]:
from bitsandbytes.optim import LAMB
def add_weight_decay(model,weight_decay = 1e-2):
    decayed = []
    no_decayed = []
    for name,param in model.named_parameters():
        if "norm" in name.lower():
            print("Skipped: ",name)
            no_decayed.append(param)
        else:
            decayed.append(param)
    return [{'params': decayed,'weight_decay': weight_decay},
            {'params': no_decayed,'weight_decay': 0}]

optimizer = LAMB(add_weight_decay(model,1e-3),max_unorm=10)

Skipped:  bert.embeddings.LayerNorm.weight
Skipped:  bert.embeddings.LayerNorm.bias
Skipped:  bert.encoder.layer.0.attention.output.LayerNorm.weight
Skipped:  bert.encoder.layer.0.attention.output.LayerNorm.bias
Skipped:  bert.encoder.layer.0.output.LayerNorm.weight
Skipped:  bert.encoder.layer.0.output.LayerNorm.bias
Skipped:  bert.encoder.layer.1.attention.output.LayerNorm.weight
Skipped:  bert.encoder.layer.1.attention.output.LayerNorm.bias
Skipped:  bert.encoder.layer.1.output.LayerNorm.weight
Skipped:  bert.encoder.layer.1.output.LayerNorm.bias
Skipped:  bert.encoder.layer.2.attention.output.LayerNorm.weight
Skipped:  bert.encoder.layer.2.attention.output.LayerNorm.bias
Skipped:  bert.encoder.layer.2.output.LayerNorm.weight
Skipped:  bert.encoder.layer.2.output.LayerNorm.bias
Skipped:  bert.encoder.layer.3.attention.output.LayerNorm.weight
Skipped:  bert.encoder.layer.3.attention.output.LayerNorm.bias
Skipped:  bert.encoder.layer.3.output.LayerNorm.weight
Skipped:  bert.encoder.la

In [30]:
# Loss functions
from bidirectional_lambda_loss import LambdaRankLoss
lambda_rank_loss_ = LambdaRankLoss(k=1000)

def regression_loss(preds, targets):
    return torch.mean((preds - targets) ** 2)


def margin_based_pairwise_loss(preds, targets, base_margin=0.1, minority_boost=0):
    preds_diff = preds.unsqueeze(1) - preds.unsqueeze(0)
    targets_diff = targets.unsqueeze(1) - targets.unsqueeze(0)
    valid_pairs = (targets_diff > 0).float()

    # Dynamic margin: Boost for pairs involving minority class
    margins = base_margin + minority_boost * (targets.unsqueeze(1) < -0.5).float()

    # Compute loss
    loss_matrix = torch.relu(margins - preds_diff) * valid_pairs
    num_pairs = valid_pairs.sum()

    if num_pairs > 0:
        return loss_matrix.sum() / num_pairs
    else:
        return torch.tensor(0.0, requires_grad=True, device=preds.device)
    
def lambda_rank_loss(preds,targets):
    return lambda_rank_loss_(preds,targets,max((targets==-1).sum(),(targets==1).sum()))
    

# Total loss
def total_loss_fn(preds, targets,classes, lambda0 = 1,lambda1=1e-2):
    reg_loss = regression_loss(preds, targets)
    
    rank_loss = margin_based_pairwise_loss(preds, targets)
    lambda_loss = lambda_rank_loss(preds,targets)
    return rank_loss*lambda1 + lambda_loss*lambda1 + reg_loss*lambda0

In [32]:
def compute_running_average(total,running_average,step,running_smooth = 0.9999):
    if running_average > 0:
        running_average_new = (running_average*step + total)/(step+1)
        
        running_average_new = running_average_new*running_smooth+(1-running_smooth)*total
    else:
        running_average_new = total
    return running_average_new

In [33]:
def log_weights(targets,n_col):
    return torch.Tensor([(len(targets)+1)/(targets[:,n_col].sum()+1)]).to('cuda')

In [34]:
from tqdm import tqdm
import torch
from sklearn.metrics import roc_auc_score
from IPython.display import clear_output
import matplotlib.pyplot as plt
import gc
import seaborn as sns

from tensorboardX import SummaryWriter
DIR = "uw_33333333llll"
writer = SummaryWriter(logdir = DIR)
bce = torch.nn.BCEWithLogitsLoss()
mse = torch.nn.MSELoss()
total_loss = 0
total_loss_ts = 0
total_loss_l = 0
total_loss_dl = 0
total_loss_s = 0
total_loss_likes = 0
total_loss_dislikes = 0

total_loss_b = 0
all_p = []
all_pld = []
all_t = []
all_pd = []
all_td = []
step = 0
model.requires_grad_(True)
model.i_feats.requires_grad_(False)


eval_size = -1
val_steps = 10000

NUM_EPOCHS =100
WARMUP = 1
LEARNING_RATE = 1e-4
EVAL_EPOCHS = 1



total_l = 0
total_dl = 0
running_average_l = 0
running_average_dl = 0
running_smooth = 0.9999


auc_l= 0
auc_dl = 0
auc_pld = 0
auc_pdl = 0


all_p = []
all_t = []
all_p_dl = []
all_t_dl = []

for e in range(NUM_EPOCHS):

    for pairs,users_meta_,items_meta_,targets,sizes in tqdm(train_loader):
        if step % 100 == 0:
            gc.collect()
            torch.cuda.empty_cache()

        miss_l = 0
        miss_dl = 0
        _auc_l= 0
        _auc_dl = 0

        likes = torch.Tensor(targets['like']).cuda()
        dislikes = torch.Tensor(targets['dislike']).cuda()

        general_targets = torch.cat((dislikes.unsqueeze(1),likes.unsqueeze(1)),dim=1).cuda()


        model.train()
        # freeze_bn(model)
        drops = np.random.randn(1000)
        drops -= drops.min()
        drops/= drops.max()
        for name,module in model.named_modules():
            if isinstance(module,nn.Dropout) and 'mask' not in name:
                if 'embedding' not in name:
                    module.p = np.clip(np.random.choice(drops)**2,0.4,0.9)
                else:
                    module.p = np.clip(np.random.choice(drops)**2,0.1,0.2)



        if e < WARMUP:
            for i in range(len(optimizer.param_groups)):
                optimizer.param_groups[i]['lr'] = LEARNING_RATE * step/(len(train_loader)*WARMUP)
        else:
            for i in range(len(optimizer.param_groups)):
                optimizer.param_groups[i]['lr'] = LEARNING_RATE - LEARNING_RATE*step/(len(train_loader))/NUM_EPOCHS



        optimizer.zero_grad()
        ts,out,dl,share,bookmarks = model(pairs,users_meta_,items_meta_)
        

        loss = 0
        loss_l = 0
        loss_s = 0
        loss_b = 0
        loss_ts = 0
        loss_like = 0
        loss_dislike = 0
        u = 0
        for u_n in sizes:
            u_out = out[u:u+u_n]
            u_targets = targets[u:u+u_n]
            l = u_out
            u_general_targets = general_targets[u:u+u_n]


            targets_ = torch.Tensor(u_targets['targets_']).cuda()
            loss_l_ = torch.mean(torch.binary_cross_entropy_with_logits(l.squeeze(1),torch.Tensor(u_targets['like']).cuda(),pos_weight = log_weights(u_targets.to_numpy(),0)))
            loss_ts_ = total_loss_fn(l.squeeze(1),targets_,torch.Tensor(u_targets['like']).cuda() - torch.Tensor(u_targets['dislike']).cuda())


            loss+= loss_ts_



            if u_general_targets[:,1].sum() == 0 or u_general_targets[:,1].sum() == len(u_general_targets[:,1]):

                _auc_l += 0.5
            else:
                _auc_l += roc_auc_score(u_general_targets[:,1].detach().cpu(),u_out.detach().cpu())

            if u_general_targets[:,0].sum() == 0 or u_general_targets[:,0].sum() == len(u_general_targets[:,0]):

                _auc_dl += 0.5
            else:
                _auc_dl += roc_auc_score(u_general_targets[:,0].detach().cpu(),u_out.detach().cpu()*-1)
            u+=u_n

            loss_l += loss_l_
            loss_ts += loss_ts_
                
        try:        
            auc_l += _auc_l/(len(sizes))
        except ZeroDivisionError:
            auc_l+=0.5/len(sizes)

        try:
            auc_dl += _auc_dl/(len(sizes))  
        except ZeroDivisionError:
            auc_dl+=0.5/len(sizes)
        

        loss_l = loss_l/len(sizes)
        loss_ts = loss_ts/len(sizes)
        
        
        total_loss += loss_ts.item()

        total_l += loss_l.item()

        running_average_l = compute_running_average(total_l,running_average_l,step)

        
        
        loss.backward()
        optimizer.step()




        if step % val_steps == val_steps-1:
            gc.collect()
            torch.cuda.empty_cache()
            print("TRAIN:_____________________________________________________________________________")
            print("timespent: ", total_loss_ts/val_steps)
            print("roc_star_loss: ", total_loss/val_steps)
            print(running_average_l)
            print(f"LR:{optimizer.param_groups[0]['lr']}")
            writer.add_scalar("train_roc_star",total_loss/val_steps,step)
            writer.add_scalar("train_running",running_average_l,step)
            writer.add_scalar("learning_rate",optimizer.param_groups[0]['lr'],step)
            writer.add_scalar("train_auc_l",auc_l/(val_steps-miss_l),step)
            writer.add_scalar("train_auc_dl",auc_dl/(val_steps-miss_dl),step)

            print(auc_l/(val_steps))     
            print(auc_dl/(val_steps))
            auc_l= 0
            auc_dl = 0
            auc_pld = 0
            auc_pdl = 0
            miss_l = 0
            miss_dl = 0

            total_loss_ts = 0
            total_loss_l = 0
            total_loss_dl = 0
            total_l_loss = 0
            total_loss_s = 0
            total_loss_b = 0
            total_loss = 0



            torch.save(model.state_dict(),f"{DIR}/cp.pt")
            torch.save(optimizer.state_dict(),f"{DIR}/optim.pt")
            gc.collect()
            torch.cuda.empty_cache()


            import seaborn as sns
            import matplotlib.pyplot as plt
            num_heads = model.config.num_attention_heads



            
            vtotal_loss_ts = 0
            vtotal_loss_l = 0
            vtotal_loss_dl = 0

            vtotal_loss = 0
            
            vtotal_l_loss = 0
            model.eval()
            vauc_l = 0
            vauc_dl = 0

            gc.collect()
            torch.cuda.empty_cache()
            with torch.no_grad():
                        for i,(pairs,users_meta_,items_meta_,targets,sizes) in enumerate(tqdm(valid_loader)):
                            vmiss_l = 0
                            vmiss_dl = 0
                    
                            
                            ts,out,dl,share,bookmarks = model(pairs,users_meta_,items_meta_)
                            
                            vloss = 0
                            vp_loss = 0
                            vl_loss = 0
                            vloss_s = 0
                            vloss_b = 0
                            vloss_like = 0
                            vloss_ts = 0
                            vloss_dislike = 0
                            u = 0
                            _vauc_l = 0
                            _vauc_dl = 0

                            for u_n in sizes:
                                u_out = out[u:u+u_n]
                                u_targets = targets[u:u+u_n]
                                l = u_out

                                targets_ = torch.Tensor(u_targets['targets_']).cuda()

                                vl_loss += total_loss_fn(l.squeeze(1),targets_,torch.Tensor(u_targets['like']).cuda() - torch.Tensor(u_targets['dislike']).cuda())

                                if u_targets[:,0].sum() == 0 or u_targets[:,0].sum() == len(u_targets[:,0]):
                                    vmiss_l +=1
                                else:
                                    _vauc_l += roc_auc_score(u_targets[:,0],u_out.detach().cpu())

                                if u_targets[:,1].sum() == 0 or u_targets[:,1].sum() == len(u_targets[:,1]):
                                    vmiss_dl +=1
                                else:
                                    _vauc_dl += roc_auc_score(u_targets[:,1],u_out.detach().cpu()*-1)
                                u+=u_n

                            vl_loss/= len(sizes)


                            try:
                                vauc_l += _vauc_l/(len(sizes)-vmiss_l)
                            except ZeroDivisionError:
                                vauc_l += 0.5/len(sizes)
                            
                            try:
                                vauc_dl += _vauc_dl/(len(sizes)-vmiss_dl)  
                            except ZeroDivisionError:
                                vauc_dl += 0.5 / len(sizes)
                            vloss= vl_loss
                            
                            
                                    
                            vtotal_loss += vloss.item()


                            if i == eval_size:
                                break

            print("VALID:_____________________________________________________________________________")
            print("timespent: ", vtotal_loss_ts/(i+1))
            print("roc_star_loss: ", vtotal_loss/(i+1))

            print(vauc_l/(i+1))
            print(vauc_dl/(i+1))
            writer.add_scalar("valid_timespent",vtotal_loss_ts/(i+1),step)
            writer.add_scalar("valid_roc_star",vtotal_loss/(i+1),step)

            writer.add_scalar("roc_auc",vauc_l/(i+1),step)
            writer.add_scalar("roc_auc_dl",vauc_dl/(i+1),step)



            
            torch.save(model.state_dict(),f"{DIR}/{e}_{vauc_l/(i+1)}.pt")
            torch.save(optimizer.state_dict(),f"{DIR}/{e}_{vauc_l/(i+1)}optim.pt")
            gc.collect()
            torch.cuda.empty_cache()

        total_l = 0 
        step+=1






 22%|██▏       | 9999/45851 [50:30<3:06:22,  3.21it/s]

TRAIN:_____________________________________________________________________________
timespent:  0.0
roc_star_loss:  0.22628476801048963
1.166189630744746
LR:2.1807594163704172e-05
0.7073949013853469
0.5043681770174929


100%|██████████| 45/45 [00:29<00:00,  1.55it/s]


VALID:_____________________________________________________________________________
timespent:  0.0
roc_star_loss:  0.2733403427733315
0.6759869507524838
0.57903790061115


 44%|████▎     | 19999/45851 [1:41:34<2:08:19,  3.36it/s]

...

TRAIN:_____________________________________________________________________________
timespent:  0.0
roc_star_loss:  0.21654434976689518
1.1717427240207592
LR:9.563806678153148e-05
0.7411477633607398
0.5053473279744377


100%|██████████| 45/45 [00:29<00:00,  1.54it/s]


VALID:_____________________________________________________________________________
timespent:  0.0
roc_star_loss:  0.27492141193813746
0.676105552697203
0.5779955852648131


 44%|████▍     | 20302/45851 [1:44:23<2:11:22,  3.24it/s] 


KeyboardInterrupt: 

In [44]:
test_pairs = pl.read_csv('test_pairs.csv')


test = test_pairs.with_columns(
    pl.col("user_id").cast(pl.UInt32),pl.col("item_id").cast(pl.UInt32)
)

In [45]:
class DS_Test(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data = data
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

In [46]:
def collate_test(data):
    data = pl.concat(data)
    pairs = data.select("user_id","item_id")
    users_meta = data.select("gender","age")
    items_meta = data.select("source_id",	"duration")
    return pairs,users_meta,items_meta

In [47]:
test = test.join(users_meta,on='user_id').join(it,on = 'item_id')
test_ds = DS_Test(test_pairs)
test_loader = DataLoader(test,shuffle=False,batch_size=1024,collate_fn=collate_test)


In [48]:
predict = None
from tqdm import tqdm
model.eval()
with torch.no_grad():
    for pairs,users_meta_,items_meta_ in tqdm(test_loader):
        if predict is None:
            predict = model(pairs,users_meta_,items_meta_)[1].squeeze().detach().cpu().numpy()
        else:
            predict = np.concatenate((predict,model(pairs,users_meta_,items_meta_)[1].squeeze().detach().cpu().numpy()))

100%|██████████| 1617/1617 [00:55<00:00, 29.12it/s]


In [49]:
predict[0]

0.14886862

In [50]:
test_pairs.with_columns(predict=predict).write_csv('sample_submission.csv')